In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the uploaded files
birth_data = pd.read_csv('csv/시군구_출생아수_full.csv')
fertile_women_data = pd.read_csv('csv/가임 여성수_full_2.csv')
education_expenses_data = pd.read_csv('csv/사교육비 총액_full.csv')
wage_data = pd.read_csv('csv/임금_full.csv')
house_price_data = pd.read_csv('csv/집값(평균)_full.csv')
marriage_data = pd.read_csv('csv/혼인건수_full.csv')

# Display the first few rows of each dataset to understand their structure
birth_data.head(), fertile_women_data.head(), education_expenses_data.head(), wage_data.head(), house_price_data.head(), marriage_data.head()

In [ ]:
# Reshape data: Convert wide-format year columns to long format
def reshape_data(df, value_name):
    return df.melt(id_vars=["AREA"], var_name="Year", value_name=value_name)

# Standardize column names for consistency
education_expenses_data.columns = education_expenses_data.columns.str.replace(',', '').str.strip()
wage_data = wage_data.rename(columns={"AGE": "AREA"}) # Rename 'AGE' column to 'AREA' in wage_data

# Reshape the data again
birth_data_long = reshape_data(birth_data, "Births")
fertile_women_long = reshape_data(fertile_women_data, "Fertile_Women")
education_expenses_long = reshape_data(education_expenses_data, "Education_Expenses")
wage_data_long = reshape_data(wage_data, "Wages")
house_price_long = reshape_data(house_price_data, "House_Price")
marriage_data_long = reshape_data(marriage_data, "Marriages")

# Merge all datasets on AREA and Year
merged_data = birth_data_long.merge(
    fertile_women_long, on=["AREA", "Year"]
).merge(
    education_expenses_long, on=["AREA", "Year"]
).merge(
    wage_data_long, on=["AREA", "Year"]
).merge(
    house_price_long, on=["AREA", "Year"]
).merge(
    marriage_data_long, on=["AREA", "Year"]
)



In [ ]:
# Display column names for each dataset to identify inconsistencies
birth_data.columns, fertile_women_data.columns, education_expenses_data.columns, wage_data.columns, house_price_data.columns, marriage_data.columns


In [ ]:
# Standardize column names for consistency
education_expenses_data.columns = education_expenses_data.columns.str.replace(',', '').str.strip()
wage_data = wage_data.rename(columns={"AGE": "AREA"})

# Reshape the data again
birth_data_long = reshape_data(birth_data, "Births")
fertile_women_long = reshape_data(fertile_women_data, "Fertile_Women")
education_expenses_long = reshape_data(education_expenses_data, "Education_Expenses")
wage_data_long = reshape_data(wage_data, "Wages")
house_price_long = reshape_data(house_price_data, "House_Price")
marriage_data_long = reshape_data(marriage_data, "Marriages")

# Merge all datasets on AREA and Year
merged_data = birth_data_long.merge(
    fertile_women_long, on=["AREA", "Year"]
).merge(
    education_expenses_long, on=["AREA", "Year"]
).merge(
    wage_data_long, on=["AREA", "Year"]
).merge(
    house_price_long, on=["AREA", "Year"]
).merge(
    marriage_data_long, on=["AREA", "Year"]
)



In [ ]:
# Check unique values of AREA and Year in each dataset for discrepancies
birth_areas = birth_data_long['AREA'].unique()
fertile_women_areas = fertile_women_long['AREA'].unique()
education_expenses_areas = education_expenses_long['AREA'].unique()
wage_areas = wage_data_long['AREA'].unique()
house_price_areas = house_price_long['AREA'].unique()
marriage_areas = marriage_data_long['AREA'].unique()

# Identify mismatches in AREA
area_sets = {
    "Birth Data": set(birth_areas),
    "Fertile Women Data": set(fertile_women_areas),
    "Education Expenses Data": set(education_expenses_areas),
    "Wage Data": set(wage_areas),
    "House Price Data": set(house_price_areas),
    "Marriage Data": set(marriage_areas),
}

# Find differences
area_differences = {name: list(area_sets["Birth Data"] - areas) for name, areas in area_sets.items()}
area_differences


In [ ]:
# Remove 'ADD_UP' from education_expenses_data
education_expenses_long = education_expenses_long[education_expenses_long["AREA"] != "ADD_UP"]

# Exclude wage_data from the merging process temporarily
merged_data = birth_data_long.merge(
    fertile_women_long, on=["AREA", "Year"]
).merge(
    education_expenses_long, on=["AREA", "Year"]
).merge(
    house_price_long, on=["AREA", "Year"]
).merge(
    marriage_data_long, on=["AREA", "Year"]
)


In [ ]:
# Clean and convert columns to numeric where necessary
merged_data["Births"] = pd.to_numeric(merged_data["Births"], errors="coerce")
merged_data["Fertile_Women"] = pd.to_numeric(merged_data["Fertile_Women"], errors="coerce")
merged_data["Education_Expenses"] = pd.to_numeric(merged_data["Education_Expenses"].str.replace(",", ""), errors="coerce")
merged_data["House_Price"] = pd.to_numeric(merged_data["House_Price"], errors="coerce")
merged_data["Marriages"] = pd.to_numeric(merged_data["Marriages"].str.replace(",", ""), errors="coerce")

# Drop rows with missing values
cleaned_data = merged_data.dropna()



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Define independent variables (X) and dependent variable (y)
X = cleaned_data[["Fertile_Women", "Education_Expenses", "House_Price", "Marriages"]]
y = cleaned_data["Births"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


In [ ]:
import matplotlib.pyplot as plt

# Plot pairwise relationships between variables to visualize correlations
pd.plotting.scatter_matrix(cleaned_data[["Births", "Fertile_Women", "Education_Expenses", "House_Price", "Marriages"]],
                           figsize=(12, 12), alpha=0.8, diagonal='hist')

plt.suptitle("Pairwise Relationships Between Variables", fontsize=16)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 실제 값 vs 예측 값 비교
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="--", label="Perfect Prediction")
plt.title("Actual vs Predicted Births")
plt.xlabel("Actual Births")
plt.ylabel("Predicted Births")
plt.legend()
plt.grid(True)
plt.show()

# 잔차 분석
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(0, color="red", linestyle="--", label="Zero Residual")
plt.title("Residuals vs Predicted Values")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.legend()
plt.grid(True)
plt.show()

# 회귀 계수 시각화
coefficients = model.coef_
features = X.columns

plt.figure(figsize=(10, 6))
plt.bar(features, coefficients, alpha=0.7, color="skyblue")
plt.title("Regression Coefficients")
plt.xlabel("Features")
plt.ylabel("Coefficient Value")
plt.grid(True, axis="y")
plt.show()